In [3]:
import pandas as pd
import psycopg2 

In [37]:
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [41]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

/var/folders/bn/6w32tfdd4wvfx46j60k7cj700000gn/T/ipykernel_1960/1731849884.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [43]:
# 1- How many schools are there in each borough?

query = """
SELECT 
    borough,
    COUNT(DISTINCT school_name) AS n_school
FROM
    nyc_schools.high_school_directory
GROUP BY
    borough
ORDER BY 
    n_school DESC;
"""

df = pd.read_sql(query, conn)
df

/var/folders/bn/6w32tfdd4wvfx46j60k7cj700000gn/T/ipykernel_1960/1370360109.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,n_school
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


In [23]:
# 2-  What is the average % of English Language Learners (ELL) per borough?

query = """
SELECT
			dir.borough AS borough,
      ROUND(AVG(dem.ell_num)::NUMERIC,2) AS avg_ell
FROM 
		nyc_schools.high_school_directory AS dir
 JOIN
		nyc_schools.school_demographics AS dem
ON dir.dbn = dem.dbn

GROUP BY 
			dir.borough
;
"""
df = pd.read_sql(query, conn)
df

/var/folders/bn/6w32tfdd4wvfx46j60k7cj700000gn/T/ipykernel_1960/1153050526.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,avg_ell
0,Manhattan,40.31


In [47]:
# 2-  top 3 schools in each borough with the highest percentage of special education students 

query = """
WITH ranked_sped_percent_school AS(
  SELECT 
			dir.borough AS borough,
      dir.school_name AS school_name,
      dem.sped_percent AS sped_percent,
      DENSE_RANK() OVER (PARTITION BY dir.borough ORDER BY dem.sped_percent DESC ) AS rank
FROM 
		nyc_schools.high_school_directory AS dir
JOIN
		nyc_schools.school_demographics AS dem
ON dir.dbn = dem.dbn
)

SELECT 
			borough,
      school_name,
      sped_percent
FROM
		ranked_sped_percent_school
WHERE 
		rank <= 3
ORDER BY 
		borough;
"""
df = pd.read_sql(query, conn)
df

/var/folders/bn/6w32tfdd4wvfx46j60k7cj700000gn/T/ipykernel_1960/3223650253.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7


In [ ]:
# because we dont have enogh data about the sped_percent and average % of English Language Learners (ELL),
# we cannot answer the secode & third  question properly about each borough . We have for the answer enough data only for borough = Manhattan.

In [35]:
# Because we still can have finding about Manhattan lets look how is the male and female distribution is in the schools.

query = """
SELECT 
			dir.borough AS borough,
      dir.school_name AS school_name,
      ROUND(AVG(dem.male_per)::NUMERIC,2) AS avg_male_per,
      ROUND(AVG(dem.female_per)::NUMERIC,2) AS avg_female_per
FROM 
		nyc_schools.high_school_directory AS dir
JOIN
		nyc_schools.school_demographics AS dem
ON dir.dbn = dem.dbn

GROUP BY 
		borough, school_name
ORDER BY
       avg_male_per DESC ;
"""
df = pd.read_sql(query, conn)
df


/var/folders/bn/6w32tfdd4wvfx46j60k7cj700000gn/T/ipykernel_1960/1625583815.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,school_name,avg_male_per,avg_female_per
0,Manhattan,Henry Street School for International Studies,57.11,42.89
1,Manhattan,University Neighborhood High School,53.01,46.99
2,Manhattan,East Side Community School,52.44,47.56
3,Manhattan,Pace High School,51.90,48.10
4,Manhattan,Marta Valle High School,50.37,49.63
5,Manhattan,"New Explorations into Science, Technology and ...",46.83,53.17
6,Manhattan,Bard High School Early College,29.40,70.60


In [ ]:
# Most Manhattan high schools show a balanced gender distribution, 
# with male and female percentages close to 50/50. However,
#Bard High School Early College stands out with a significant female majority (70.6%), 
# indicating a possible program or enrollment focus that attracts more female students.